## 1. Import libraries and datasets

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

dataset = pd.read_csv('Churn_Modelling.csv')

# CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## 2. Encode Categorical data

Label Encode the "Gender" column and One Hot Encode the "Geography" column

In [50]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Gender encode
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

# Geography column encode
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
# Geography (3), CreditScore, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary
x = np.array(ct.fit_transform(x))

## 3. Preparing the dataset into the Training set and Test set

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## 4. Create the ANN model

In [5]:
model = tf.keras.models.Sequential()

# L1: Adds the input layer and first hidden layer
model.add(tf.keras.layers.Dense(units=6, activation='relu'))

# L2: Adds the second hidden layer.
model.add(tf.keras.layers.Dense(units=6, activation='relu'))

# L3: Adds the output layer.
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile with the descent algorithm and loss function
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## 5. Train the ANN model

In [6]:
model.fit(x_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6024 - accuracy: 0.6933
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4730 - accuracy: 0.7977
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4565 - accuracy: 0.7983
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4462 - accuracy: 0.7995
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4372 - accuracy: 0.8046
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4279 - accuracy: 0.8124: 0s - loss: 0.4114 - 
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4182 - accuracy: 0.8220
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4084 - accuracy: 0.8254
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3992 - accuracy: 0.8296
Epoch 10/100
250/250 [==============================] - 0s 1ms/step -

## 6a. Predict the following observation

What is the probability this customer would churn with these given details?

| Field | Value |
| -- | -- |
| Geography | France |
| Credit Score | 600 |
| Gender | Male |
| Age (years) | 40 |
| Tenure (years) | 3 |
| Balance (\$) | 60000 |
| Number of Products | 2 |
| Have a credit card? | Yes |
| Active member? | Yes |
| Estimated Salary (\$) | 50000 |

In [53]:
print('Gender labels:\n', pd.DataFrame(le.classes_, columns=['Index to Value']), '\n')
print('Geography labels:\n', pd.DataFrame(ct.transformers_[0][1].categories_[0], columns=['Index to Value']), '\n')

observation = [
    # Geography
    1, 0, 0,
    # CreditScore
    600,
    # Gender
    1,
    # Age
    40,
    # Tenure
    3,
    # Balance
    60000,
    # Number of Products
    2,
    # Have a credit card?
    1,
    # Active member?
    1,
    # Estimated Salary
    50000
]

observation_pred = model.predict(sc.transform([observation]))[0][0]

print('Customer: ', observation, '\n')
print(f'Churn probability: {observation_pred} ({observation_pred > 0.5})')

Gender labels:
   Index to Value
0         Female
1           Male 

Geography labels:
   Index to Value
0         France
1        Germany
2          Spain 

Customer:  [1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000] 

Churn probability: 0.04363596439361572 (False)


## 6b. Predict the test dataset

In [64]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_pred = model.predict(x_test) > 0.5
cm = confusion_matrix(y_test, y_pred)

print(pd.DataFrame(cm, columns=['Churned', 'Not Churned'], index=['Pred. Churned', 'Pred. Not Churned']), '\n')
print(f'Accuracy score: {accuracy_score(y_test, y_pred)} (Correct: {cm[0][0] + cm[1][1]}, Incorrect: {cm[0][1] + cm[1][0]})')

                   Churned  Not Churned
Pred. Churned         1510           85
Pred. Not Churned      189          216 

Accuracy score: 0.863 (Correct: 1726, Incorrect: 274)
